# Beat the ATS - Project Description

###### According to Jobscan, 99% of Fortune 500 companies use an Applicant Tracking System (ATS) as their recruitment strategy.
###### All ATS' store candidates' information by using keywords, so it's easy to parse and filter them.
###### If applicant's resume does not meet certain criteria, it is either flagged or auto-rejected.
###### "Beat the ATS" Project aims to analyse the most popular tools and skills (keywords) for Data Analytics field and check if there is a relationship between those skills and earnings.

## Research Questions:
<b>All research questions are based on years 2020-2021 in the United States of America</b>
<br><br>
###### 1. What were the most popular technologies employers sought in Data Analytics field in the period 2020-2021? (Frequency analysis)
###### 2. What were the most popular tools employers sought? (Frequency analysis)
###### 3. What were the most popular soft skills? (Frequency analysis)
###### 4. Is there a relationship between education level and earnings? If yes, what is the relationship? (Regression analysis)
###### 5. Is there a relationship between years of experience and earnings? If yes, what is the relationship? (Regression analysis)
<br><br>
### Hypotheses:
###### 1. There are certain technologies that are more sought after than others.
###### 2. There are certain tools that are more sought after than others.
###### 3. There are certain soft skills that are sought after by employers.
###### 4. There is a correlation between education level and earnings.
###### 5. There is a correlation between experience level (measured in years) and earnings.


In [59]:
# Import Dependencies
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [60]:
# Declare variables and import the data
# job_descriptions = pd.read_csv("Job descriptions.csv")
# salary_education_experience = pd.read_csv("salary vs education vs experience.csv")

In [61]:
# Clean the data
# Drop N/A where relevant (not relevant in the context of education!!!)
# Words spelled differently, aggregate words for the same meaning
# Address encoding problems: convert all job descriptions to UTF-8 using unicode_escape, pandas ignore encoding)

In [62]:
# Searching for keywords:
# Stop words removal
# Count the frequency using CountVectorizer
# Produce counts
# Define words matrix
def words_matrix(words, vectorizer):
    matrix = vectorizer.fit_transform(words)

In [70]:
text = ['Hello, my my my name is Rita and and I am a data scientist.']
text2 = ['This is a vectorizer test']

In [71]:
vec = CountVectorizer()
x = vec.fit_transform(text)
print(x)

  (0, 3)	1
  (0, 5)	3
  (0, 6)	1
  (0, 4)	1
  (0, 7)	1
  (0, 1)	2
  (0, 0)	1
  (0, 2)	1
  (0, 8)	1


In [ ]:
# Create a visualisation for keywords

In [ ]:
# Calculate Measures of Central Tendency - mean, median, mode for education
# example_array = np.array([24, 16, 12, 10, 12, 28, 38, 12, 28, 24])
# example_mode = stats.mode(example_array)
# If there are multiple modes, the stats.mode() function will always return the smallest mode in the dataset.

In [ ]:
# Create a visualisation for education

In [ ]:
# Calculate Measures of Central Tendency - mean, median, mode for experience

In [ ]:
# Create a visualisation for experience

In [ ]:
# Calculate Measures of Central Tendency - mean, median, mode for earnings

In [ ]:
# Create a visualisation for earnings

In [ ]:
# Perform regression analysis for education and earnings

In [ ]:
# Create a visualisation for regression analysis: education vs earnings

In [ ]:
# Perform regression analysis for experience and earnings

In [ ]:
# Create a visualisation for regression analysis: experience vs earnings

In [ ]:
# We can define the Student t-test as a method that tells us how significant the differences can be between different groups.
# A Student t-test is defined as a statistic and this is used to compare the means of two different populations.
# t-test: stats.ttest_1samp(X, mean)
# Independent t-test: stats.ttest_ind(X,Y)

In [ ]:
# Create a visualisation for t-test